```
--- Day 10: Balance Bots ---

You come upon a factory in which many robots are zooming around handing small microchips to each other.

Upon closer examination, you notice that each bot only proceeds when it has two microchips, and once it does, it gives each one to a different bot or puts it in a marked "output" bin. Sometimes, bots take microchips from "input" bins, too.

Inspecting one of the microchips, it seems like they each contain a single number; the bots must use some logic to decide what to do with each chip. You access the local control computer and download the bots' instructions (your puzzle input).

Some of the instructions specify that a specific-valued microchip should be given to a specific bot; the rest of the instructions indicate what a given bot should do with its lower-value or higher-value chip.

For example, consider the following instructions:

value 5 goes to bot 2
bot 2 gives low to bot 1 and high to bot 0
value 3 goes to bot 1
bot 1 gives low to output 1 and high to bot 0
bot 0 gives low to output 2 and high to output 0
value 2 goes to bot 2
Initially, bot 1 starts with a value-3 chip, and bot 2 starts with a value-2 chip and a value-5 chip.
Because bot 2 has two microchips, it gives its lower one (2) to bot 1 and its higher one (5) to bot 0.
Then, bot 1 has two microchips; it puts the value-2 chip in output 1 and gives the value-3 chip to bot 0.
Finally, bot 0 has two microchips; it puts the 3 in output 2 and the 5 in output 0.
In the end, output bin 0 contains a value-5 microchip, output bin 1 contains a value-2 microchip, and output bin 2 contains a value-3 microchip. In this configuration, bot number 2 is responsible for comparing value-5 microchips with value-2 microchips.

Based on your instructions, what is the number of the bot that is responsible for comparing value-61 microchips with value-17 microchips?

```

In [1]:
import re
import numpy as np
from collections import defaultdict

RE_VALUE = re.compile("value ([0-9]+) goes to bot ([0-9]+)")
RE_GIVES = re.compile("bot ([0-9]+) gives low to (bot|output) ([0-9]+) and high to (bot|output) ([0-9]+)")

class Bot:
    
    def __init__(self):
        self.microchips = set()
        self.compared = []
        self.low = None
        self.high = None
    
    def add_microchip(self, microchip):
        self.microchips.add(microchip)
        self.update()
        
    def set_low_and_high(self, low, high):
        self.low = low
        self.high = high
        self.update()
    
    def update(self):
        if len(self.microchips) > 1 and self.low is not None and self.high is not None:
            self.low.add_microchip(min(self.microchips))
            self.high.add_microchip(max(self.microchips))
            self.compared.append(self.microchips)
            self.microchips = set()            
            

def process_instructions(instructions, verbose=False):
    
    bots = defaultdict(Bot)    
    for i, instruction in enumerate(instructions, start=1):
        instruction = instruction.strip()
        
        if verbose:
            print({k: v.microchips for k, v in bots.items()})
        
        m = RE_VALUE.match(instruction)
        if m is not None:
            value, bot = m.groups()
            bots[int(bot)].add_microchip(int(value))
            continue
            
        m = RE_GIVES.match(instruction)
        if m is not None:
            bot, low_type, low_num, high_type, high_num = m.groups()
            low = bots[int(low_num) if low_type=='bot' else -int(low_num)-1]
            high = bots[int(high_num) if high_type=='bot' else -int(high_num)-1]
            bots[int(bot)].set_low_and_high(low, high)
            continue
            
    return bots

In [2]:
example = [
    'value 5 goes to bot 2', 
    'bot 2 gives low to bot 1 and high to bot 0',
    'value 3 goes to bot 1',
    'bot 1 gives low to output 1 and high to bot 0',
    'bot 0 gives low to output 2 and high to output 0',
    'value 2 goes to bot 2'
]

In [3]:
bots = process_instructions(example, verbose=True)
print({k: v.microchips for k, v in bots.items()})

for num, bot in bots.items():
    if {2, 5} in bot.compared:
        print("\n Bot {} is comparing value-2 and value-5".format(num))

{}
{2: {5}}
{0: set(), 1: set(), 2: {5}}
{0: set(), 1: {3}, 2: {5}}
{0: set(), 1: {3}, 2: {5}, -2: set()}
{0: set(), 1: {3}, 2: {5}, -2: set(), -3: set(), -1: set()}
{0: set(), 1: set(), 2: set(), -2: {2}, -3: {3}, -1: {5}}

 Bot 2 is comparing value-2 and value-5


In [4]:
with open('inputs/day10.txt', 'rt') as instructions:
    bots = process_instructions(instructions)

for num, bot in bots.items():
    if {61, 17} in bot.compared:
        print("Bot {} is comparing value-61 and value-17".format(num))

Bot 161 is comparing value-61 and value-17


```
--- Part Two ---

What do you get if you multiply together the values of one chip in each of outputs 0, 1, and 2?
```

In [5]:
np.product(list(bots[-1].microchips | bots[-2].microchips | bots[-3].microchips))

133163